In [1]:
import pylab as pl
import pandas as pd
import numpy as np
%pylab inline
import json
import os
import scipy.stats
import math
import requests
import StringIO
import zipfile
import geopandas as gp
import statsmodels.api as sm
import urllib2
import re
import statsmodels.formula.api as smf

Populating the interactive namespace from numpy and matplotlib


In [2]:
listings = pd.read_csv('listings.csv')
listings.columns

Index([u'id', u'name', u'host_id', u'host_name', u'neighbourhood_group',
       u'neighbourhood', u'latitude', u'longitude', u'room_type', u'price',
       u'minimum_nights', u'number_of_reviews', u'last_review',
       u'reviews_per_month', u'calculated_host_listings_count',
       u'availability_365'],
      dtype='object')

In [3]:
#bnb_man_nearstops are the listings that are near to one or more MTA stations
bnb_man_nearstops = pd.read_csv('bnb_man_nearstops.txt', sep=';')
bnb_man_nearstops.columns

Index([u'FID', u'FID_man_st', u'FID_nyctsu', u'stop_id', u'stop_code',
       u'stop_name', u'stop_desc', u'stop_lat', u'stop_lon', u'zone_id',
       u'stop_url', u'location_t', u'parent_sta', u'FID_nybb', u'BoroCode',
       u'BoroName', u'Shape_Leng', u'Shape_Area', u'BUFF_DIST', u'FID_bnb_ma',
       u'FID_bnb_li', u'id', u'name', u'host_id', u'host_name', u'neighbourh',
       u'neighbou_1', u'latitude', u'longitude', u'room_type', u'price',
       u'minimum_ni', u'number_of_', u'last_revie', u'reviews_pe',
       u'calculated', u'availabili', u'FID_nybb_1', u'BoroCode_1',
       u'BoroName_1', u'Shape_Le_1', u'Shape_Ar_1'],
      dtype='object')

In [4]:
#Df1 is my subset of bnb_man_nearstops data with the relevant columns (excluding the mta station attributes)
list_of_drops=['FID','FID_man_st', 'FID_nyctsu', 'stop_id', 'stop_code','stop_name', 'stop_desc', 'stop_lat', 'stop_lon', 'zone_id','stop_url', 'location_t', 'parent_sta','FID_nybb', 'BoroCode','BoroName', 'Shape_Leng', 'Shape_Area', 'BUFF_DIST', 'FID_bnb_ma','FID_bnb_li', 'FID_nybb_1', 'BoroCode_1','BoroName_1', 'Shape_Le_1', 'Shape_Ar_1']
df1=bnb_man_nearstops.drop(list_of_drops,1)
#df4 is the same as df1, but im eliminating duplicate rows (airbnb rooms near to multiple subway stations)
#safety_indicator is a column which assigns 1 to listings near to a mta station. 
df4 = df1.drop_duplicates(subset='id', take_last=True)
df4['location_indicator']=1
#I will keep as df2 the list of just the id and the safety_indicator of the listings. 
list_of_drops=['name', 'host_id', 'host_name','neighbourh', 'neighbou_1','latitude','longitude', 'room_type', 'price', 'minimum_ni','number_of_', 'last_revie', 'reviews_pe','calculated','availabili']
df2=df4.drop(list_of_drops,1)    

C:\Users\Marco\Anaconda\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
#bnb_br_nearstops are the listings that are near to one or more MTA stations in Brooklyn
bnb_br_nearstops = pd.read_csv('bnb_br_nearstops.txt', sep=';')

In [6]:
#Df5 is my subset of bnb_br_nearstops data with the relevant columns (excluding the mta station attributes)
list_of_drops=['FID','FID_br_sto', 'FID_nybb', 'stop_id', 'stop_code','stop_name', 'stop_desc', 'stop_lat', 'stop_lon', 'zone_id','stop_url', 'location_t', 'parent_sta','FID_nybb', 'BoroCode','BoroName', 'Shape_Leng', 'Shape_Area', 'BUFF_DIST', 'FID_bnb_br','FID_bnb_li', 'FID_nybb_1', 'BoroCode_1','BoroName_1', 'Shape_Le_1', 'Shape_Ar_1']
df5=bnb_br_nearstops.drop(list_of_drops,1)
#df6 is the same as df5, but im eliminating duplicate rows (airbnb rooms near to multiple subway stations)
#safety_indicator is a column which assigns 1 to listings near to a mta station. 
df6 = df5.drop_duplicates(subset='id', take_last=True)
df6['location_indicator']=1
#I will keep as df7 the list of just the id and the safety_indicator of the listings. 
list_of_drops=['name', 'host_id', 'host_name','neighbourh', 'neighbou_1','latitude','longitude', 'room_type', 'price', 'minimum_ni','number_of_', 'last_revie', 'reviews_pe','calculated','availabili','FID_nyctsu']
df7=df6.drop(list_of_drops,1)  

C:\Users\Marco\Anaconda\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
#Now, I wanna concatenate my two data sets with the indicator: df2 and df7
mta_indicator=pd.concat([df2,df7])
#and I will merge with the original listings data frame in order to get the result
listings_tranps_indicator = pd.merge(mta_indicator,listings,how='right',on=['id'],
                                               sort=True,suffixes=('_x', '_y')  )
print "mta_indicator", len(mta_indicator), "listings", len(listings), "merge", len(listings_tranps_indicator)

listings_tranps_indicator['location_indicator'].fillna(0, inplace=True)

listings_tranps_indicator.to_csv('listings_location_indicator.csv')

mta_indicator 17786 listings 27469 merge 27469


In [ ]:
listings_tranps_indicator